In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics
!pip install supervision

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import supervision as sv
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import csv
from datetime import datetime
from shapely.geometry import Polygon, box  # To calculate areas of intersection


In [ ]:
# Load the model
model = YOLO('/content/drive/MyDrive/Model/training_final_upgrade2/weights/best.pt')

In [ ]:
# Path to the video
video_path = '/content/the best.mp4'

# Capture the video
cap = cv2.VideoCapture(video_path)

In [ ]:
# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Video dimensions: {width}x{height}")
print(f"FPS: {fps}")
print(f"Total frames: {total_frames}")



In [ ]:
# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 files
output_path = '/content/the best_pred.mp4' # Path where the output video will be saved
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

In [ ]:
# Define the callback function for inference
def callback(image: np.ndarray) -> sv.Detections:
    result = model(image)[0]
    return sv.Detections.from_ultralytics(result)

# Define the slicer object with the callback function
slicer = sv.InferenceSlicer(callback=callback)

In [ ]:
# Get a frame from the video
ret, frame = cap.read()


#ROI
roi_points = np.array([[400, 300], [340, 500], [720, 500], [540, 300]], np.int32)
roi_points = roi_points.reshape((-1, 1, 2))  # Reshape for polylines


cv2.polylines(frame, [roi_points], isClosed=True, color=(255, 255, 0), thickness=3)  # Draw ROI in white

# Display the frame with the ROI
plt.figure(figsize=(10, 10)) #Removed extra indentation
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
plt.title("Initial Frame with ROI")
plt.show()



#Counting

In [ ]:
# Define your ROI for counting vehicles
right_roi = np.array([[400, 300], [340, 500], [720, 500], [540, 300]], np.int32)
right_roi_polygon = Polygon(right_roi.reshape(-1, 2))  # Create a Shapely polygon for the ROI

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get video FPS
fps = cap.get(cv2.CAP_PROP_FPS)
frame_interval = 1

frame_count = 0
heavy_frame_counter = 0
medium_or_flow_counter = 0

# CSV file setup
csv_file = open('traffic_data.csv', mode='w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Exit', 'City','Street','Timestamp','Vehicles Count', 'Traffic State', 'Gate State'])

# Gate state variable
gate_state = "Open"

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if frame_count % frame_interval == 0:
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        # Perform YOLOv8 inference on the frame
        sliced_detections = slicer(image=frame)

        if isinstance(sliced_detections, tuple):
            sliced_detections = sv.Detections(*sliced_detections)

        results = model(frame)
        cv2.polylines(frame, [right_roi], True, (255, 255, 255), 2)

        vehicles_in_roi_count = 0

        for result in results:
            for i in range(len(sliced_detections.xyxy)):
                x1, y1, x2, y2 = map(int, sliced_detections.xyxy[i])
                confidence = sliced_detections.confidence[i]
                class_id = sliced_detections.class_id[i]
                class_name = model.names[class_id]

                # Bounding box as a shapely box object
                box_polygon = box(x1, y1, x2, y2)

                # Calculate the intersection between the box and the ROI
                intersection_area = right_roi_polygon.intersection(box_polygon).area

                if intersection_area > 0:
                    # Calculate percentage of bounding box inside the ROI
                    percentage_inside = (intersection_area / box_polygon.area) * 100

                    # Only count the vehicle if more than 50% of its area is inside the ROI
                    if percentage_inside > 50:
                        vehicles_in_roi_count += 1

                        # Draw bounding box and label
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 255), 2)
                        label = f'{class_name} {confidence:.2f}'
                        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # Determine traffic state based on the number of vehicles in the ROI
        if vehicles_in_roi_count >= 20:
            traffic_state = 'Heavy'
            heavy_frame_counter += 1
            medium_or_flow_counter = 0  # Reset the counter for medium/flow
        elif vehicles_in_roi_count >= 15:
            traffic_state = 'Medium'
            heavy_frame_counter = 0  # Reset heavy frame counter
            medium_or_flow_counter += 1  # Increment the medium/flow counter
        else:
            traffic_state = 'Flow'
            heavy_frame_counter = 0  # Reset heavy frame counter
            medium_or_flow_counter += 1  # Increment the medium/flow counter

        # Gate state logic
        if heavy_frame_counter >=2:
            gate_state = 'Closed'  # The gate closes if heavy for 60 frames
        elif medium_or_flow_counter >= 2:
            gate_state = 'Open'  # The gate opens if medium or flow for 60 frames

        # Write to CSV with City included
        csv_writer.writerow(['Exit 9', 'Riyadh','Al Hamra',timestamp, vehicles_in_roi_count, traffic_state, gate_state])

        # Display results on frame
        height, width, _ = frame.shape
        cv2.putText(frame, f"Vehicles in ROI: {vehicles_in_roi_count}", (10, height - 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        cv2.putText(frame, f"Gate State: {gate_state}", (10, height - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        cv2.putText(frame, f"Time: {timestamp}", (10, height - 90), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        cv2.putText(frame, f"Traffic State: {traffic_state}", (10, height - 120), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # Display frame
        cv2_imshow(frame)
        out.write(frame)

    frame_count += 1

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
csv_file.close()
